Before Pruning" and "After Layer-wise Pruning" accuracy values are too low (0.02% and 0.06%), which suggests a major issue.
maybe
Issue: Incorrect Model Output Dimension
Issue: Incorrect Softmax Activation Before Evaluation
Issue: Pruning Removed Too Many Weights
Issue: Data Normalization is Incorrect

In [ ]:
# ✅ STEP 1: Install Dependencies
!pip install torch torchvision transformers torchsummary datasets tabulate

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.2.0 requires fsspec==2025.2.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install --upgrade gcsfs fsspec


  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.2.0-py3-none-any.whl (184 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2025.2.0 which is incompatible.


In [ ]:
# ✅ STEP 2: Import Libraries
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchsummary import summary
from tabulate import tabulate

In [ ]:
# ✅ STEP 3: Load Pre-trained Model (ResNet-50 or BERT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Vision Model (ResNet-50)
model = torchvision.models.resnet50(pretrained=True)
model.to(device)
model.eval()
summary(model, (3, 224, 224))

# NLP Model (BERT)
from transformers import BertForSequenceClassification
nlp_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
nlp_model.to(device)
nlp_model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# ✅ STEP 4: Load Dataset (CIFAR-10 for Vision / IMDB for NLP)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# ✅ STEP 5: Evaluation Function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"🔥 Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
# ✅ STEP 6: Test Accuracy Before Pruning
print("🚀 Evaluating model BEFORE pruning:")
accuracy_before = evaluate_model(model, testloader)

🚀 Evaluating model BEFORE pruning:
🔥 Accuracy: 0.02%


In [ ]:
# ✅ STEP 7: Apply Layer-wise Pruning (From Paper [13])
def layerwise_pruning(model, sparsity=0.5):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=sparsity)

layerwise_pruning(model, sparsity=0.5)


In [ ]:
# ✅ STEP 8: Test Accuracy After Layer-wise Pruning
print("🚀 Evaluating model AFTER layer-wise pruning:")
accuracy_after_layerwise = evaluate_model(model, testloader)

🚀 Evaluating model AFTER layer-wise pruning:
🔥 Accuracy: 0.06%


In [ ]:
# ✅ STEP 9: Apply N:M Structured Pruning (From Paper [15])
def structured_pruning(tensor, N=2, M=4):
    shape = tensor.shape
    tensor = tensor.view(-1, M)
    mask = torch.zeros_like(tensor)
    topk_indices = torch.topk(torch.abs(tensor), N, dim=1, largest=True).indices
    mask.scatter_(1, topk_indices, 1)
    return (tensor * mask).view(shape)

for name, param in model.named_parameters():
    if len(param.shape) > 1:
        model.state_dict()[name] = structured_pruning(param, N=2, M=4)

In [ ]:
# ✅ STEP 10: Test Accuracy After N:M Structured Pruning
print("🚀 Evaluating model AFTER N:M structured pruning:")
accuracy_after_nm = evaluate_model(model, testloader)

🚀 Evaluating model AFTER N:M structured pruning:
🔥 Accuracy: 0.06%


In [ ]:
# ✅ STEP 11: Fine-tuning the Pruned Model (Recover Performance)
def fine_tune(model, trainloader, epochs=3, learning_rate=0.001):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

fine_tune(model, trainloader, epochs=3)

Epoch 1, Loss: 0.5561
Epoch 2, Loss: 0.3209
Epoch 3, Loss: 0.2474


In [ ]:
# ✅ STEP 12: Test Accuracy After Fine-Tuning
print("🚀 Evaluating model AFTER fine-tuning:")
accuracy_finetuned = evaluate_model(model, testloader)

🚀 Evaluating model AFTER fine-tuning:
🔥 Accuracy: 85.53%


In [ ]:
# ✅ STEP 13: Save the Pruned Model
torch.save(model.state_dict(), "pruned_model.pth")
print("✅ Pruned model saved successfully!")


✅ Pruned model saved successfully!


In [ ]:
# ✅ STEP 14: Display Comparison Table
data = [
    ["Before Pruning", accuracy_before],
    ["After Layer-wise Pruning", accuracy_after_layerwise],
    ["After N:M Pruning", accuracy_after_nm],
    ["After Fine-Tuning", accuracy_finetuned]
]

print("\n🔥 Performance Comparison:")
print(tabulate(data, headers=["Stage", "Accuracy (%)"], tablefmt="github"))


🔥 Performance Comparison:
| Stage                    |   Accuracy (%) |
|--------------------------|----------------|
| Before Pruning           |           0.02 |
| After Layer-wise Pruning |           0.06 |
| After N:M Pruning        |           0.06 |
| After Fine-Tuning        |          85.53 |
